# Zero to Hero

## 1.11 Arbol training/testing

El objetivo es calcular la ganancia en testing de un arbol de decision

In [82]:
rm( list=ls())
gc()

library( "data.table")   #cargo la libreria  data.table
library( "rpart")  #cargo la libreria  rpart
library( "rpart.plot")
library( "caret")

options(repr.plot.width=20, repr.plot.height=10) 
setwd("C:/_MCD/Labo1/code")


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2299972,122.9,4508395,240.8,3094765,165.3
Vcells,3908024,29.9,97606996,744.7,123105904,939.3


In [83]:
dataset <- fread("./datasets/competencia1_2022.csv")   #cargo el dataset #cargo el dataset
dataset <- dataset[ foto_mes==202101 ]  # me quedo solo con el periodo 202101

In [84]:
#creo training con 70%  y testing con 30%
set.seed(13)
train_rows <- createDataPartition(dataset$clase_ternaria, p= 0.70,list= FALSE)
dtrain  <- dataset[ train_rows]
dtest  <-  dataset[ -train_rows]

Entreno el modelo en los datos de training

In [85]:
#param  <- list("cp"= -0.5,
#               "minsplit"=  900,
#               "minbucket"= 440,
#               "maxdepth"= 5
#              )
param  <- list( "cp"=         -.9,  #complejidad minima
                 "minsplit"=  653,  #minima cantidad de registros en un nodo para hacer el split
                 "minbucket"= 317,  #minima cantidad de registros en una hoja
                 "maxdepth"=    7 ) #profundidad máxima del arbol


Entreno el modelo en los datos de training

In [86]:
#genero el modelo
modelo <-  rpart::rpart(formula= "clase_ternaria ~ ." ,
                        data= dtrain,
                        xval= 0,
                        control= param)

aplico el modelo a los datos de testing

In [87]:
prediccion  <- predict( modelo, dtest, type = "prob")

### Calculo la ganancia del modelo en los datos de testing

a partir de la prediccion, calculo la probabilidad de BAJA+2 de cada registro de testing  **dtest**

In [88]:
prob_baja2  <- prediccion[, "BAJA+2"]

Para el cálculo de la ganancia del modelo aplicado a testing debo tener en cuenta lo siguiente
* Solo envio estimulo a los registros que el modelo asigno una probabilidad mayor a 0.025
* Si no envio estimulo, no gano ni pierdo, es decir la ganacia es cero
* Si el registro al que envio el estímulo es un **BAJA+2**  entonces la ganancia es de +78000
* Si el registro al que envio el estímulo es un **BAJA+1**  entonces la ganancia es de  -2000
* Si el registro al que envio el estímulo es un **CONTINUA** entonces la ganancia es de  -2000

##### A la condición que solo envio a los registros con probabilidad mayor a 1/60 la resuelvo con la condicion lógica  (prob_baja2> 0.025)
<br>
Esta condicion termina siendo un 0 o un 1
<br>
vale 1 si y solo si se cumple la condicion, es decir  si prob_baja2 es mayor a 0.025

La ganancia de +78000 o -2000 la resuelvo de esta forma ifelse( clase_ternaria=="BAJA+2", 78000, -2000)

Finalmente,  que se cumplan ambas condiciones al mismo tiempo, en este caso, es multiplicarlas, o sea
<br>
(prob_baja2> 0.025) * ifelse( clase_ternaria=="BAJA+2", 78000, -2000)
<br>
La condicion anterior toma los siguientes valores

| prob(BAJA+2) | clase_ternaria | ganancia |
| --- | :---: | ---: |
| menor o igual a 0.025 | BAJA+1 | 0 |
| menor o igual a 0.025 | CONTINUA | 0 |
| menor o igual a 0.025 | BAJA+2 | 0 |
| mayor a 0.025 | BAJA+1 | -2000 |
| mayor a 0.025 | CONTINUA | -2000 |
| mayor a 0.025 | BAJA+2 | 78000 |


Finalmente hago el calculo en data.table

In [89]:
ganancia_testing <- dtest[ , sum(  (prob_baja2>0.025) * ifelse( clase_ternaria=="BAJA+2", 78000, -2000) )]

In [90]:
ganancia_testing

[1] 4678000

Es importante que la ganancia esta calculada sobre los datos de testing, que en este caso son apenas el 30%
<br>
Si quiero extrapolar a todo el dataset, debo hacer el **cociente** de esa ganancia por 0.30   ( coloquialmente, debo dividir por 0.30 )

In [91]:
ganancia_testing_normalizada  <-  ganancia_testing/0.3

In [92]:
ganancia_testing_normalizada

[1] 15593333

Ahora calculo lo mismo para train y comparo

In [93]:
prediccion  <- predict( modelo, dtrain, type = "prob")
prob_baja2  <- prediccion[, "BAJA+2"]
ganancia_train <- dtrain[ , sum(  (prob_baja2>0.025) * ifelse( clase_ternaria=="BAJA+2", 78000, -2000) )]
ganancia_train_normalizada  <-  ganancia_train/0.7
print(paste0("Test: ", round(ganancia_testing_normalizada)," VS. Train: ", round(ganancia_train_normalizada)))

[1] "Test: 15593333 VS. Train: 26642857"
